---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}


In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    #Read in txt file, which is a list of cities and their respective colleges
    univ_town = pd.read_fwf('university_towns.txt', header=None)
    univ_town.columns = ['College']
    #Create a blank state column, will be used later
    univ_town['State'] = None
    #The list includes state entries, and they all end with '[edit]' since it is pulled from wikipedia
    #Use this loop to remove that part and place it into the state column
    for x in range(len(univ_town)-1):
        if univ_town.loc[x, 'College'][-6:] == '[edit]':
            univ_town.loc[x, 'State'] = univ_town.loc[x, 'College'][:-6]
    #This will forward fill all blank states with the last nonblank state
    #So all colleges after Alabama and before Alaska will be filled with Alabama
    univ_town['State'].ffill(inplace=True)
    #Now since entries with just states don't correspond to a college town, we can remove those entries
    index_drop = univ_town[univ_town['College'].str[-6:] == '[edit]'].index
    univ_town.drop(index_drop, inplace=True)
    #Split the college string based on ( because it is formatted as college_town (university)
    #only want to keep the town so use .str[0]
    univ_town['College'] = univ_town['College'].str.split('(').str[0]
    #Remove leading white space
    univ_town['College'] = univ_town['College'].str.rstrip()
    #Reaname columns
    univ_town.columns = ['RegionName', 'State']
    #Reset index
    univ_town.reset_index(inplace=True)
    #Create a list of ordered pairs using the state first and then the college town second
    univ_town = univ_town[['State', 'RegionName']]
    univ_town
    
    return univ_town

In [4]:
get_list_of_university_towns().head(10)

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', skiprows=5)
    #Want to focus on these two columns and rename them
    gdp = gdp[['Unnamed: 4',
           'GDP in billions of chained 2009 dollars.1']]
    gdp.columns =['Quarter', 'GDP']
    gdp.dropna(inplace=True)
    #Initialize a year column to split quarter string into
    #Need to split so we can filter based on year
    gdp['Year']=''
    #quarter column is formatted as yyyyqx so split string on the q into year and quarter columns
    gdp['Year'], gdp['Quarter'] = gdp['Quarter'].str.split('q').str[0], gdp['Quarter'].str.split('q').str[1]
    #Focus on years after 2000
    gdp = gdp[gdp['Year'].astype(int) >= 2000]
    gdp.reset_index(inplace=True, drop=True)
    #Now that we filtered years, reformat quarter column to include year and quarter
    for x in range(len(gdp['Quarter'])):
        gdp.loc[x, 'Quarter'] = '{}q{}'.format(gdp.loc[x, 'Year'], gdp.loc[x, 'Quarter'])
    gdp = gdp[['Quarter', 'GDP']]
    gdp['GDP'] = pd.to_numeric(gdp['GDP'])
    #Recession is defined as two consecutive quarters of GDP contraction
    two_qs_in_row = []
    
    #i+2 is the focus quarter
    #i->i+1 must increase to indicate recession was not already active
    #i+1->i+2 must decrease to indicate start of recession
    #i+2->i+3 must also decrease because it must be 2 consecutive quarters of contraction
    #Append the focus quarter to the list to indicate the start of a recession
    for i in range(len(gdp)-3):
        if (gdp.loc[i, 'GDP'] < gdp.loc[i+1, 'GDP']) \
        & (gdp.loc[i+1, 'GDP'] > gdp.loc[i+2, 'GDP']) \
        & (gdp.loc[i+2, 'GDP'] > gdp.loc[i+3, 'GDP']):
            two_qs_in_row.append(gdp.loc[i+2, 'Quarter'])
    return two_qs_in_row[0]

In [6]:
get_recession_start()

'2008q3'

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    #All code up until the break is the same as last to format the dataframe
    gdp = pd.read_excel('gdplev.xls', skiprows=5)
    gdp = gdp[['Unnamed: 4',
           'GDP in billions of chained 2009 dollars.1']]
    gdp.columns =['Quarter', 'GDP']
    gdp.dropna(inplace=True)
    gdp['Year']=''
    gdp['Year'], gdp['Quarter'] = gdp['Quarter'].str.split('q').str[0], gdp['Quarter'].str.split('q').str[1]
    gdp = gdp[gdp['Year'].astype(int) >= 2000]
    gdp.reset_index(inplace=True, drop=True)
    for x in range(len(gdp['Quarter'])):
        gdp.loc[x, 'Quarter'] = '{}q{}'.format(gdp.loc[x, 'Year'], gdp.loc[x, 'Quarter'])
    gdp = gdp[['Quarter', 'GDP']]
    gdp['GDP'] = pd.to_numeric(gdp['GDP'])
    
    
    two_qs_in_row = []
    end_rec = []
    #First snippet of code does the same as last, where it finds the recession start
    #Because a recession can't end unless it has already started
    #Focus is now on i+4
    #Looking for two consecutive quarters of gdp growth after a recession start
    for i in range(len(gdp)-4):
        if (gdp.loc[i, 'GDP'] < gdp.loc[i+1, 'GDP']) \
        & (gdp.loc[i+1, 'GDP'] > gdp.loc[i+2, 'GDP']) \
        & (gdp.loc[i+2, 'GDP'] > gdp.loc[i+3, 'GDP']):
            two_qs_in_row.append(gdp.loc[i+2, 'Quarter'])
        if two_qs_in_row:
            if (gdp.loc[i, 'GDP'] > gdp.loc[i+1, 'GDP']) \
            & (gdp.loc[i+1, 'GDP'] > gdp.loc[i+2, 'GDP']) \
            & (gdp.loc[i+2, 'GDP'] < gdp.loc[i+3, 'GDP']) \
            & (gdp.loc[i+3, 'GDP'] < gdp.loc[i+4, 'GDP']):
                end_rec.append(gdp.loc[i+4, 'Quarter'])
    answer = end_rec[0]   
    return answer

In [8]:
get_recession_end()

'2009q4'

In [9]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    #Code up until break is the same as before to format dataframe
    gdp = pd.read_excel('gdplev.xls', skiprows=5)
    gdp = gdp[['Unnamed: 4',
           'GDP in billions of chained 2009 dollars.1']]
    gdp.columns =['Quarter', 'GDP']
    gdp.dropna(inplace=True)
    gdp['Year']=''
    gdp['Year'], gdp['Quarter'] = gdp['Quarter'].str.split('q').str[0], gdp['Quarter'].str.split('q').str[1]
    gdp = gdp[gdp['Year'].astype(int) >= 2000]
    gdp.reset_index(inplace=True, drop=True)
    for x in range(len(gdp['Quarter'])):
        gdp.loc[x, 'Quarter'] = '{}q{}'.format(gdp.loc[x, 'Year'], gdp.loc[x, 'Quarter'])
    gdp = gdp[['Quarter', 'GDP']]
    gdp['GDP'] = pd.to_numeric(gdp['GDP'])
    two_qs_in_row = []
    
    
    
    bottom = ''
    for i in range(len(gdp)-4):
        if (gdp.loc[i, 'GDP'] < gdp.loc[i+1, 'GDP']) \
        & (gdp.loc[i+1, 'GDP'] > gdp.loc[i+2, 'GDP']) \
        & (gdp.loc[i+2, 'GDP'] > gdp.loc[i+3, 'GDP']):
            two_qs_in_row.append(gdp.loc[i+2, 'Quarter'])
        if two_qs_in_row:
            if (gdp.loc[i, 'GDP'] > gdp.loc[i+1, 'GDP']) \
            & (gdp.loc[i+1, 'GDP'] > gdp.loc[i+2, 'GDP']) \
            & (gdp.loc[i+2, 'GDP'] < gdp.loc[i+3, 'GDP']) \
            & (gdp.loc[i+3, 'GDP'] < gdp.loc[i+4, 'GDP']):
                bottom = gdp.loc[i+2, 'Quarter']
    return bottom

In [10]:
get_recession_bottom()

'2009q2'

In [11]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    #Values in this dataframe will be mean housing prices in each quarter
    
    
    house = pd.read_csv('City_Zhvi_AllHomes.csv')
    clean_cols = ['State', 'RegionName']
    #Focus on year columns
    year_cols = house.columns[6:]
    year_to_keep = []
    #split up month and year to filter out years before 2000
    for x in year_cols:
        year, month = x.split('-')
        if int(year) >= 2000:
            year_to_keep.append(x)
    cols_to_keep = clean_cols + year_to_keep
    #Now our columns are State, RegionName, and columns for month/year after 2000
    house = house[cols_to_keep]
    clean_year = []
    #Divide each month into quarters
    for x in year_to_keep:
        year, month = x.split('-')
        if int(month) <= 3:
            x = year + 'q1'
        elif (int(month) > 3) & (int(month) <= 6):
            x = year + 'q2'
        elif (int(month) > 6) & (int(month) <= 9):
            x = year + 'q3'
        else:
            x = year + 'q4'
        clean_year.append(x)
    clean_cols += clean_year
    house.columns = clean_cols
    #Replace state abbreviations with state names using list in first cell of project
    house['State'] = house['State'].replace(states)
    #Set MultiIndex using State and RegionName
    house.set_index(['State', 'RegionName'], inplace=True)
    #Group the columns, using the mean of each quarter, so we're left with only one column per quarter
    answer = house.groupby(by=house.columns, axis=1).mean()
    
    return answer

In [12]:
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [13]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    #Get our list of university towns
    univ_towns = get_list_of_university_towns()
    #Set this column equal to 1 to indicate university town, when merged, non-univ towns will be 0
    univ_towns['univ_town'] = 1
    #Set index so that it is the same for all datasets
    univ_towns.set_index(['State', 'RegionName'], inplace=True)
    
    #Get our housing data into a dataframe and merge the dataframes, keeping all data from the housing dataframe
    data = convert_housing_data_to_quarters()
    combined = data.merge(univ_towns, how='left', left_index=True, right_index=True)
    #Fill all blank univ_town entries with 0 to indicate not a university town
    combined['univ_town'].fillna(0, inplace=True)
    start = get_recession_start()
    bottom = get_recession_bottom()
    #Our dataframe y will now have mean housing price in each town at the start of the recession, and at the peak of the recession
    #And also whether or not it is a university town
    y = combined[[start, bottom, 'univ_town']]
    #Ratio of peak recession prices to start recession prices
    y['change'] = y[bottom]/y[start]
    
    #Two series, one containing the ratio of prices for university towns and the other for non-university towns
    univ_town = y[y['univ_town'] == 1]['change'].dropna()
    non_univ = y[y['univ_town'] == 0]['change'].dropna()
    
    #Run a ttest to see if there is a statistically significant difference between the two 
    t = ttest_ind(univ_town, non_univ)
    #Save the p-value for the ttest
    p = t[1]
    
    #Set different to False and change to True if results are statistically significant
    different = False
    if p < 0.01:
        different =True
        
    #Set better to whichever category produced statistically significant better results
    better = 'university town'
    if univ_town.mean() < non_univ.mean():
        better = 'non-university town'
    return different, p, better

In [14]:
run_ttest()

C:\Users\Tyler\anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(True, 0.00897558754071226, 'university town')

Null Hypothesis: University towns have their mean housing prices affected the same amount as non-University towns

With a p-value of 0.0090, we can reject the null hypothesis and conclude that University towns have their mean housing prices less effected by recessions.